In [2]:
import numpy as np # Matrix와 vector와 같은 Array 연산을 할 때 사용하는 라이브러리
import os #운영체제와의 상호작용을 돕는 다양한 기능을 제공하는 모듈
import cv2 # openCV 사용할때는 cv2로 import함
import matplotlib.pyplot as plt #다양한 데이터를 많은 방법으로 도식화 할 수 있도록 하는 파이썬 라이브러리
from tqdm import tqdm # 즉석에서 progress bar를 생성해주고 함수나 반복문의 TTC (Time To Completion) 를 예측하는 파이썬 패키지
import time 
from PIL import Image #이미지 처리와 그래픽기능 제공하는 라이브러리 (픽셀 단위의 조작,마스킹,투명도 제어,이미지 필터)
import random  # 파이썬에서 난수(random number)를 구할 수 있는 모듈 
import openslide # digital pathology에서 whole slide image를 읽기 위한 간단한 인터페이스를 제공하는 라이브러리
import re # 파이썬에서 정규 표현식을 지원하는 모듈
import math # 수학과 관련된 다양한 함수들과 상수들이 미리 정의되어 있는 모듈
from skimage.morphology import skeletonize # skeletonize : reduces binary objects to 1 pixel wide representations
import xml.etree.ElementTree as ET # XML 데이터를 구문 분석하고 만들기 위한 단순하고 효율적인 API(프로그램들이 서로 상호작용하는 것을 도와주는 매게체)를 구현하는 모듈.
import tifffile # Tifffile is a Python library to
                # 1.store numpy arrays in TIFF (Tagged Image File Format) files, and (numpy array를 TIFF파일로 저장)
                # 2.read image and metadata from TIFF-like files used in bioimaging. (바이오이미징에서 아용되는 파일의 메타데이터 읽는 것)

import skimage #  scikit-image는 이미지처리에 특화된 Python 이미지 라이브러리이며 Numpy배열로 이미지 객체를 다룸.
from skimage import morphology
from mpl_toolkits.axes_grid1 import make_axes_locatable #axes.grid1 :matplotlib로 (여러) 이미지를 쉽게 표시할 수 있는 도우미 클래스 모음

import copy # 리스트 래퍼런스로 인한 문제를 해결하기 위해 사용가능

#PyTorch 는 여러분이 신경망(neural network)를 생성하고 학습시키는 것을 도와주기 위해서
# torch.nn , torch.optim , Dataset , 그리고 DataLoader 와 같은 잘 디자인된 모듈과 클래스들을 제공합니다.

import torch # facebook에서 제공하는 딥러닝 도구로서, numpy와 효율적인 연동을 지원하는 편리한 도구이다
import torch.nn as nn # 신경망을 생성할 수 있는 패키지
import torch.nn.functional as F # torch.nn.functional로 구현한 함수의 경우에는 인스턴스화 시킬 필요 없이 사용이 가능(nn과의 차이)
import torch.optim as optim # 다양한 optimization 알고리즘을 구현한 패키지
import torchvision # PyTorch와 함께 사용되는 Computer Vision 용 라이브러리
import torchvision.transforms as trasnforms # common image transformations

import timm # 유명한 네트워크 불러오는 라이브러리(현재사용하고 있는 네트워크도 이것으로 불러옴) (수업때는 torch hub에서 불러왔었음)
import segmentation_models_pytorch as smp 

import albumentations as A #fast and flexible image augmentation library.
from albumentations.pytorch import ToTensorV2 # Convert image and mask to torch.Tensor and divide by 255
                                               #(if image or mask are uint8 type.)
import h5py # HDF4 HDF5는 매우크고 복잡한 대용량 데이터를 저장하기 위한 파일형식으로 h5py 패키지는 HDF5 바이너리 데이터 형식에 대한 Pythonic 인터페이스이다.

In [3]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    random.seed(seed_value) 
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars\n
        torch.backends.cudnn.deterministic = True  #needed\n
        torch.backends.cudnn.benchmark = False

In [14]:
seed = 42
random_seed(seed,True)

psize = (224,224) # patch size??

# level 0 : 400배 (한 pixel 당 0.1945 마이크로미터) LN_001.mrxs 같은 경우 pixel width, pixel height 모두 0.3890 마이크로미터 였음
# level 1 : 200배
# level 2 : 100배
# level 3 : 50배
# level 4 : 25배

level_3 = 3 # level 3 :  50배
level_1 = 1 # level 1 : 200배
overlap = (192,192) # patch 와 patch간의 서로 ovelap되는 영역의 크기 ???
batch_size = 100
threshold = 0.8 # cancer일 probability?? patch내에서 foreground의 영역의 크기??

# 학습된 parameter의 경로
model_path = {
    'level_3' : "Y:\\hsyoo\\pth\\pth_80\\level_3\\best_model_level3.pth",
    'level_1' : "hi"
}

#  GPU 사용하기 위해 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [15]:
# ToTensorV2 Convert image and mask to torch.Tensor and divide by 255



val_imtrans = A.Compose([ # 데이터 로드 과정
    A.Normalize(), # 정규화
    ToTensorV2()
])

In [16]:
# whole slide image를 읽고 모델에 넣고 

def read_wsi_clf(slide, level, psize, overlap, net): # slide, level, patch사이즈?? ,patch끼리 overap되는 영역사이즈, 모델을 인수로 받음
    max_x = slide.level_dimensions[level][0] # x축에 대한 max 값??
    max_y = slide.level_dimensions[level][1] # y축에 대한 max 값??
    num = 0
    
    
    out_fill_3 = np.zeros(psize) # patch size의 0으로 만 이루어진 넘파이 어레이 생성
    out_fill_2 = np.zeros((psize[0]//2,psize[1]//2)) # ??? (outfill_3의 넓이의 1/4)
    out_fill_1 = np.zeros((psize[0]//4, psize[1]//4)) #??? (outfill_3의 넓이의 1/16)
    if level == 3 or level == 4:   # level이 3이거나 4면 
        overlay = np.zeros([max_y, max_x]) # overlay와 reference에 max_y max_x 크기의 원소 0인 넘파이 어레이 저장
        reference = np.zeros([max_y, max_x])
    elif level == 2:                # level 2이면
        overlay = np.zeros([max_y//2, max_x//2]) # overlay와 reference에 max_y//2 max_x//2 크기의 원소 0인 넘파이 어레이 저장
        reference = np.zeros([max_y//2, max_x//2]) 
    elif level == 1:                 # level 1이면
        overlay = np.zeros([max_y//4, max_x//4]) # overlay와 reference에 max_y//4 max_x//4 크기의 원소 0인 넘파이 어레이 저장
        reference = np.zeros([max_y//4, max_x//4])

    #ceil() 함수는 소수점 자리의 숫자를 무조건 올리는 함수이다.
    # 만약 max_x - opverap이 10이고 max_x - opverap이 2면 step_x는 5
    steps_x = int(math.ceil((max_x - overlap[0]) / float(psize[0] - overlap[0]))) # 총 x좌표축에 대한 step의 수 계산??
    steps_y = int(math.ceil((max_y - overlap[1]) / float(psize[1] - overlap[1]))) # 총 y좌표축에 대한 step의 수 계산??
    start_coords = list() # start 좌표?? start point??
    
    for y in tqdm(range(0,steps_y)): # 0~step_y까지 반복문 실행
        col_list = list()
        
        for x in range(0, steps_x): # 0~step_x 까지 반복문 실행
            x_start = x*(psize[0] - overlap[0]) # sliding 과정에서 각 과정의 x좌표에 대한 왼쪽 끝 좌표??
            x_end = x_start + psize[0] # sliding 과정에서 각 과정의 x 좌표에 대한 오른쪽 끝 좌표??
            y_start = y*(psize[1] - overlap[1]) # sliding 과정에서 각 과정의 y좌표에 대한 위쪽 끝 좌표??
            y_end = y_start + psize[1]  # sliding 과정에서 각 과정의 y 좌표에 대한 아래쪽 끝 좌표??
            
            if (x_end > max_x):  # x_end가 슬라이드의 max_x값보다 크다면
                x_overlap = x_end - max_x # x_end에서 max_x값 빼서 x_overlap 이라는 변수에 저장
                x_start = max_x - psize[0] # 패치의 x축에 대한 시작 좌표 = max_x - 패치사이즈의 x좌표축에 대한 길이
                x_end = max_x  # 패치의 x좌표축에서의 오른쪽 끝 좌표
            if(y_end > max_y): # y_end가 슬라이드의 max_y값보다 크다면
                y_start = max_y - psize[1] # 패치 사이즈의 y축에 대한 길이만큼 뺸 것을 시작 점으로
                y_end = max_y # 끝점은 max_y로 설정
            
            # ???
            temp = np.array(slide.read_region((x_start*(2**(level)), y_start*(2**(level))), level, psize)) # 패치의 x축에 대한 중심좌표와 y축에 대한 중심좌표와 레벨 패치사이즈에 대한 것을 numpy array로 만듬
            print(temp)
            temp = temp[:,:,:3] # 알파 채널에 대한 array만 사용?
            print(temp)
            temp = temp.astype(np.uint8) # uint8로 data type 변경
            print(temp)
            temp = val_imtrans(image = temp)['image'].unsqueeze(0).to(device).float() # 이미지 변환하고 gpu사용????
            #언스퀴즈(Unsqueeze) - 특정 위치에 1인 차원을 추가한다 .unsqueeze(0) 첫번째 차원 추가
            print(temp)
            
            
            if level == 3 or level ==4: #  level이 3이거나 4면
                start_coords.append((y_start,x_start)) # start 좌표 리스트에 (y_start,x_start) 추가
            elif level == 2:            #  level이 2 이면
                start_coords.append((y_start//2,x_start//2)) #start 좌표 리스트에 각 x,y start에 대해서 나누기 2를 해준 튜플을 추가
            elif level == 1:
                start_coords.append((y_start//4,x_start//4)) ##start 좌표 리스트에 각 x,y start에 대해서 나누기 4을 해준 튜플을 추가
            
            """
            batch size로 쌓기
            """
            # torch.cat()은 주어진 차원을 기준으로 주어진 텐서들을 붙입(concatenate)니다.
            # cat함수는 concatenate를 해주는 함수이고 concatenate하고자 하는 차원을 증가시킨다 (차원의 수는 유지된다). 
            # concatenate하고자하는 차원을 지정해주면 그 차원으로 두 tensor의 차원을 더한 값으로 차원이 변경된다.
            
            if num==0:  # 0이면
                cat = temp 
                cat = torch.cat((cat,temp), dim = 0) # cat과 temp라는 두 tesor를 첫번째 차원에 대해서 concatenate 
            num+=1
            
            """
            model에 배치로 넣기
            """
            if num == batch_size:  # num 이 batch size와 같다면
                out = net(cat)     # cat이라는 tensor를 모델에 넣는 것???  # net 이라는 것의 출력은 어떤 것인지??
                out = F.softmax(out, dim = 1) # torch.nn.functional()의 F로 output의 두 번째 dimension에 대해 softmax적용
                out = out.detach().cpu().numpy() # gpu가 아닌 cpu를 사용하는것 ??????? 
                out = out[:,1] # ???????????
                if level == 3 or level == 4: # 만약 level이 3이거나 4면
                    for i in range(batch_size): # batch size만큼 반복문 실행
                        out_fill_3.fill(out[i]) # 0으로만 이루어진 numpy array를 ouput들의 결과로 값을 채운다?? 밑에 과정 이해안감 (reference와 overlay 둘 다 max_y max_x 크기의 원소 0인 넘파이 어레이)
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] += out_fill_3
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] +=1
                        if i ==batch_size-1: #batch size까지 다 돌았으면
                            num=0            # num 초기화 
                            start_coords = [] # start좌표(point??) 초기화
                elif level == 2:  # 만약 level이 2면 
                    for i in range(batch_size): # batch size만큼 반복문 실행(level2인 경우)
                        out_fill_2.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] += out_fill_2
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] +=1
                        if i ==batch_size-1:
                            num=0
                            start_coords = []
                elif level == 1: #만약 level이 1이면
                    for i in range(batch_size): # batch size만큼 반복문 실행(level1인 경우)
                        out_fill_1.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] += out_fill_1
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] +=1
                        if i ==batch_size-1:
                            num=0
                            start_coords = []
            elif y==steps_y-1 and x==steps_x-1: # x와 y가 y step의 개수와 x step의 개수와 같다면
                out = net(cat)        # cat이라는 tensor를 모델에 넣는 것???  # net 이라는 것의 출력은 어떤 것인지??
                out = F.softmax(out, dim = 1) 
                out = out.detach().cpu().numpy()
                out = out[:,1]
                if level == 3 or level == 4:
                    for i in range(num):
                        out_fill_3.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] += out_fill_3
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] +=1
                elif level ==2:
                    for i in range(num):
                        out_fill_2.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] += out_fill_2
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] +=1
                elif level ==1:
                    for i in range(num):
                        out_fill_1.fill(out[i])
                        overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] += out_fill_1
                        reference[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] +=1

    output = overlay / reference # overlay는 += out_fill_3, reference는 += 1
    return output 


In [6]:
a= []        
a = ['a','b','c','d','e','f','g']
#print(a[1:3,-1])
#b= [[1,2,3],[4,5]]
#print(b[:,1])

In [17]:
net = timm.create_model("tf_efficientnet_b0_ns", pretrained = True, num_classes = 2) # 모델 정의
net.load_state_dict(torch.load(model_path['level_3'])) # level3에 대한 모델 사용? 
net = net.to(device).eval()  #모델 evaluation 하기 위한 상태로 바꿈?? (효율적인 메모리? 연산 ? 위해)

In [18]:
root_dir = "Y:\\data\\"  # data에 대한 경로  
modes = ["valid"]          
meta_labels = ["NOLN_metastasis", "LN_metastasis"] 
wsi_fns = list() # filename 저장할 list 생성??
include_num = [] # (저장할 때 파일명에 들어가는) 숫자들을 저장하기 위한 빈 리스트 생성??
for i in range(66,101): # 66 부터 100까지 반복문 실행  
    num = format(i, '03') # 세자리 숫자 만들고 세자리 미만일 경우는 앞에 0붙여서 formatting하는 것
    include_num.append(num) # 리스트에 앞에서 formatting된 숫자들을 append
for mode in modes:# 이 코드에서는 valid mode만존재
    for meta_label in meta_labels: # NOLN 다음에 LN
        for patient in sorted(os.listdir(os.path.join(root_dir, mode, meta_label))): ##root_dir+mode+meta_label에 있는 모든 파일과 폴더명 리스트를 반환
            if patient.split('.')[-1] == 'mrxs' and patient.split('.')[0].split('_')[1] in include_num: # 하나씩 불러 뽑았을 때 그것이 mrxs(슬라이드이미지)이고 66~100 번째 환자에 대한 슬라이드에 해당한다면 
                wsi_fns.append(os.path.join(root_dir, mode, meta_label, patient)) # 그것들을 wsi_fns라는 리스트에 해당 경로들을 추가 

In [19]:
magnifications_40x = ["LN_068", "LN_069", "LN_070", "LN_071", "LN_072", "LN_073", "LN_074", "LN_075", "LN_076", "LN_077", 
"LN_078", "LN_079", "LN_080", "LN_081", "LN_082", "LN_083", "LN_084", "LN_085", "LN_086", "LN_087", 
"LN_088", "LN_089", "LN_090", "LN_091", "LN_092", "LN_093", "LN_094", "LN_095", "LN_096", "LN_097",
"LN_098",  "LN_099", "LN_100", "NOLN_067", "NOLN_068", "NOLN_069", "NOLN_070", "NOLN_071", "NOLN_072", "NOLN_073", "NOLN_074", 
"NOLN_075", "NOLN_076", "NOLN_077", "NOLN_078", "NOLN_079", "NOLN_080", "NOLN_081", "NOLN_082", 
"NOLN_083", "NOLN_084", "NOLN_085", "NOLN_086", "NOLN_087", "NOLN_088", "NOLN_089", "NOLN_090", 
"NOLN_091", "NOLN_092", "NOLN_093", "NOLN_094", "NOLN_095", "NOLN_096", "NOLN_097", "NOLN_098", 
"NOLN_099", "NOLN_100"] # 400배 확대한 것들에 해당하는 slide들을 저장한 리스트

for wsi_fn in wsi_fns:       # 슬라이드에 대한 file name들을 하나씩 가져와서
    name = wsi_fn.split('/')[-1].split('.')[0] # 경로명에서 파일이름만 취하고 거기서 확장자를 뺀 파일이름만 들어가도록
    print(name)               # 파일 이름 출력
    if name in magnifications_40x: # 만약 이름안에 magnifications_40x가 있다면 ????? level_0이 400배가 아닌지
        level_3 = 4  # level3에 4를 넣고
    else:
        level_3 = 3  # 그렇지 않다면 level3에 3넣음
        
##############################################################################################################
                    # mask_save_dir = "/mnt/pathology/hsyoo/h5/train/ensemble"
                    #(os.path.join(mask_save_dir, f"{name}.h5"), 'w')
        
    f = h5py.File(f'Y:\\hsyoo\\h5\\heatmap_step2\\{name}.h5', 'w') # h5포맷의 파일을 쓰기모드로 불러옴
    slide = openslide.OpenSlide(wsi_fn)      # openslide로 열어서 slide에 저장하고
    print(f"slide.level_dimensions : {slide.level_dimensions}, apply level : {level_3}") # 슬라이드 level의 차원
    overlay_3 = read_wsi_clf(slide, level_3, psize, overlap, net) # clf??  ## 관심 있는 connected component를 포함하는 배열
    print(f"overlay_3 shape : {overlay_3.shape}") #?
    print(f"overlay_3 min max : {overlay_3.min()}, {overlay_3.max()}") # 
    f['level_3'] = overlay_3 # 파일에서 level_3에 해당하는 곳에 overlay_3을 저장
    
    """
    여기 위에 overlay추가 가능, 밑부터는 저장단계
    """
    d=copy.deepcopy(overlay_3)    # mutable 한 내부객체(내부리스트)의 문제를 해결하기 위해 deepcopy사용
    overlay_3[overlay_3 >= threshold] =1 # overlay3이 threshold인 0.8보다 크다면 1
    overlay_3[overlay_3 < threshold] = 0 # overlay3이 threshold인 0.8보다 작다면 0
    overlay_3 = overlay_3.astype(bool) # overlay3의 데이타타입을 bool로 바꿈

    # ~_object : 지정된 크기보다 작은 object를 제거합니다. ar는 레이블이 지정된 객체가 있는 배열일 것으로 예상하고 min_size보다 작은 객체를 제거합니다.
    #            If ar is bool, the image가 먼저 표시된다(is first labeled)
    #            (ar, min_size=64, connectivity=1, in_place=False, *, out=None)              
    #             min_size : 허용되는 가장 작은 object 크기, connectivity : 픽셀의 neighborhood를 정의 하는 connectivity (ar이 bool이라면 labeling중에 사용) 
    #             ar :  interest object를 포함하는 배열입니다. 배열 type이 int이면 int는 음수가 아니어야 함              
    b = morphology.remove_small_objects(overlay_3, 50000, connectivity=8) 
    
    # ~_holes : 지정된 크기보다 작은 contiguous 구멍을 제거합니다.
    # remove_small_holes(ar, area_threshold=64, connectivity=1, in_place=False, *, out=None)
    # ar : 관심 있는 connected component를 포함하는 배열입니다.
    # area_threshold : 채워질 contiguous hole의 최대 면적(픽셀)입니다. min_size 를 대체합니다 .
    c = morphology.remove_small_holes(b, 50000, connectivity=8)   
    
    slide = np.array(slide.read_region((0,0),level_3,slide.level_dimensions[level_3]))[:,:,:3]  #slide.read_region **********
    
    f['post'] = c # post-processing 
    """
    visualization
    """
    fig = plt.figure(dpi = 200)
    ax = fig.add_subplot(1,4,1)
    plt.imshow(slide)    # slide 이미지를 출력
    plt.axis('off')
    plt.title(f"{name}")


    ax = fig.add_subplot(1,4,2)   # 슬라이드 이미지위에 overay3를 겹쳐 출력 
    plt.imshow(slide)  
    plt.imshow(overlay_3,cmap='jet', alpha = 0.5) # 투명도 0.5
    plt.title('Prediction') 
    plt.axis('off')
    
    ax = fig.add_subplot(1,4,3) # 슬라이드 이미지위에 post-processing된 결과를 겹쳐서 출력
    plt.imshow(slide)
    plt.imshow(c,cmap='jet', alpha = 0.5) 
    plt.axis('off')
    plt.title('Post-process')

    ax = fig.add_subplot(1,4,4) #  슬라이드 d?? 
    im = plt.imshow(d, vmin = 0.5, vmax = 1.0, cmap = 'jet')
    plt.title('heatmap')
    plt.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax) # colorbar 출력
    plt.tight_layout() # colorbar나 title들이 서로 겹치치 않도록
    plt.show()
    fig.savefig(f"Y:\\mhson\\step1_predict\\heatmap_80\\{name}_l3.png") # 지정된 경로에 저장 ( savefig() 함수를 사용해서 그래프를 이미지 파일 등으로 저장)
    plt.close(fig) 
    """
    위까지는 저장단계, 밑부터는 후처리 단계! -> 후처리는 bool형태로 이루어져야하므로 Thresholding이 발생한다!
    자세한 후처리 사항들은 check_segmentation_80의 remove_small_object, remove_small_holes를 통해 진행하자.
    step2 aug짜는건 후처리 전 threshold값으로 미리 짜둘 수 있다. bool로 바꾸면 어차피 똑같
    """
    # ~~~
    f.close()

Y:\data\valid\NOLN_metastasis\NOLN_066


OSError: Unable to create file (unable to open file: name = 'Y:\hsyoo\h5\heatmap\Y:\data\valid\NOLN_metastasis\NOLN_066.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)